# HPE Redfish Ansible playbooks

Version 0.116

## Introduction

The HPE Ansible Redfish [Gitub repository](https://github.com/HewlettPackard/ansible-ilorest-role) proposes several types of Ansible Redfish Playbook examples.

In this Jupyter Notebook, you will study the Ansible Redfish playbooks based upon the HPE Redfish Python library [examples](https://github.com/HewlettPackard/python-ilorest-library/tree/master/examples/Redfish).

TBD




## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation (Version: 0.13) ############

# Set Student ID number
stdid={{ STDID }}
Id=$(id --user --name)
NbId=3
InvFile=${NbId}/hosts

# location and ports variables
IloSyBasePort={{ ILOSYBASEPORT }}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

# Fake Credentials as we are testing against a BMC simulator
OvSsoToken="FakeOvSsoToken"

# Miscellaneous
WorkshopDir=$PWD
HpePythonRedfishVenv="${NbId}/HpePythonRedfishVenv"
export PYTHONPATH="${WorkshopDir}/${NbId}/library/"
w=$(basename $PWD)

alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo "Good News: $bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Create the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${iLO5SimulatorIP} ansible_port={{ ILOSYBASEPORT }}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}
# Below is a fake session token as we are testing against an iLO 5 simulator
token="${OvSsoToken}"
__EOF__


## TBD

In [ ]:
# Create Virtual Python environment (Venv)
[ -d ${HpePythonRedfishVenv} ] && rm -r ${HpePythonRedfishVenv} &>/dev/null 
python3 -m venv ${HpePythonRedfishVenv}                         &>/dev/null
source ${HpePythonRedfishVenv}/bin/activate                     &>/dev/null

# Populate Python Venv with the HPE python-ilorest-library
pip install wheel                                               &>/dev/null
pip install python-ilorest-library                              &>/dev/null

# Install latest Ansible in the Venv
pip install ansible                                             &>/dev/null

echo -e "\n\n\tFinished creating Venv\n\n"

In [ ]:
# Retrieve Indicator LED status
python3 ${NbId}/library/get_uid_light.py

In [ ]:
# Launch Ansible Playbook to toggle Indicator LED
ansible-playbook -i ${InvFile} ${NbId}/set_uid_light.yml

In [ ]:
# Retrieve Indicator LED status
python3 ${NbId}/library/get_uid_light.py

## Reset iLO 5 simulator

In [ ]:
ResetSimulators